<h1 align="center">Lab 2:  Sexism Identification in Twitter</h1>
<h2 align="center">Session 5. Large Language Models: Prompting and In-context Learning</h2>
<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Mateusz Król
- (replace with another name)

### CONSTANTS

In [ ]:
COLAB = True # IF YOU USE GOOGLE COLAB -> COLAB = True
PIP = False # IF YOU NEED INSTALL LIBRARIES -> PIP = True

## Install some libraries

In [ ]:
if PIP:
    !pip install transformers --upgrade
    !pip install datasets accelerate
    !pip install evaluate
    !pip install -U PyEvALL

In [ ]:
#!pip install torch
#!pip install numpy
#!pip install pandas
#!pip install scikit-learn

In [ ]:
import os
import sys
from torch.utils.data import Dataset, DataLoader
import torch


# if False:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from typing import List, Tuple
#from datasets import Dataset
import tqdm

## Import readerEXIST2025 library, and read the dataset

In [ ]:
if COLAB is True:
  from google.colab import drive
  drive.mount('/content/drive')
  base_path = "/content/drive/MyDrive/LNR/"
else:
  base_path = "../"


Mounted at /content/drive


In [ ]:
library_path = os.path.join(base_path, "Project/")
sys.path.append(library_path)
from readerEXIST2025 import EXISTReader

In [ ]:
# path to the dataset, adapt this path wherever you have the dataset
dataset_path = os.path.join(base_path, "Project/EXIST 2025 Tweets Dataset/")

file_train = os.path.join(dataset_path, "training/EXIST2025_training.json")
file_dev = os.path.join(dataset_path, "dev/EXIST2025_dev.json")
file_test = os.path.join(dataset_path, "test/EXIST2025_test_clean.json")


reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test)


EnTrainTask1, EnDevTask1, EnTestTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1"), reader_test.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2, EnTestTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2"), reader_test.get(lang="EN", subtask="2")
EnTrainTask3, EnDevTask3, EnTestTask3 = reader_train.get(lang="EN", subtask="3"), reader_dev.get(lang="EN", subtask="3"), reader_test.get(lang="EN", subtask="3")


SpTrainTask1, SpDevTask1, SpTestTask1  = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1"), reader_test.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2, SpTestTask2  = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2"), reader_test.get(lang="ES", subtask="2")
SpTrainTask3, SpDevTask3, SpTestTask3  = reader_train.get(lang="ES", subtask="3"), reader_dev.get(lang="ES", subtask="3"), reader_test.get(lang="ES", subtask="3")


In [ ]:
# !ls ../../corpora/EXIST_2025_Dataset_V0.3/

## Dataset class

In [ ]:
class SexismDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)
        }

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.2 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import emoji

# Regular expressions for cleaning
web_re = re.compile(r"https?:\/\/[^\s]+", re.U)  # Remove URLs
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)  # Remove mentions
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)  # Remove hashtags
special_chars_re = re.compile(r"[^a-zA-Z0-9\s]", re.U)  # Remove special characters

def preprocess(text):
    text = pd.Series(text)  # Convert input to Pandas Series if it's not already

    text = text.str.replace(user_re, "", regex=True)  # Remove @mentions
    text = text.str.replace(web_re, "", regex=True)  # Remove URLs
    text = text.str.replace(hashtag_re, "", regex=True)  # Remove hashtags
    text = text.apply(lambda x: emoji.replace_emoji(x, replace=""))  # Remove emojis
    text = text.str.replace(special_chars_re, "", regex=True)  # Remove special characters
    text = text.str.lower().str.strip()  # Convert to lowercase and remove extra spaces

    return text  # Convert back to a list if needed

In [ ]:
EnTrainTask1 = (EnTrainTask1[0], preprocess(EnTrainTask1[1]), EnTrainTask1[2])
EnDevTask1 = (EnDevTask1[0], preprocess(EnDevTask1[1]), EnDevTask1[2])
EnTrainTask2 = (EnTrainTask2[0], preprocess(EnTrainTask2[1]), EnTrainTask2[2])
EnDevTask2 = (EnDevTask2[0], preprocess(EnDevTask2[1]), EnDevTask2[2])

In [ ]:
SpTrainTask1 = (SpTrainTask1[0], preprocess(SpTrainTask1[1]), SpTrainTask1[2])
SpDevTask1 = (SpDevTask1[0], preprocess(SpDevTask1[1]), SpDevTask1[2])
SpTrainTask2 = (SpTrainTask2[0], preprocess(SpTrainTask2[1]), SpTrainTask2[2])
SpDevTask2 = (SpDevTask2[0], preprocess(SpDevTask2[1]), SpDevTask2[2])

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random
import os
import pandas as pd
import sys
import tempfile
import time

In [ ]:
def set_seed(seed=1234):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
def compute_metrics_1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def compute_metrics_2(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def sexism_classification_pipeline_task1(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=2, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc= LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_1,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    # If there is a test dataset
    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task1.csv', index=False)
        print("Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv")
        return model, submission_df
    return model, eval_results

def sexism_classification_pipeline_task2(trainInfo, devInfo, testInfo=None, model_name='bert-base-uncased', nlabels=3, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc= LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        #metric_for_best_model=args.get('metric_for_best_model',"ICM")
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_2,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    # If there is a test dataset
    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)

        })
        submission_df.to_csv('sexism_predictions_task2.csv', index=False)
        print("Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv")
        return model, submission_df
    return model, eval_results

### Task 1

In [ ]:
set_seed()
# modelname
modelname = "cardiffnlp/twitter-roberta-base-2022-154m"
# training params
params = {
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "num_train_epochs": 10
}
# pipeline for the subtask1
model, submission_df = sexism_classification_pipeline_task1(EnTrainTask1, EnDevTask1, EnTestTask1, modelname,
2, "single_label_classification", **params )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.676200,0.670852,0.563063,0.000000,0.000000,0.000000
2,0.460900,0.370554,0.840090,0.822943,0.797101,0.850515
3,0.297800,0.370688,0.844595,0.824427,0.814070,0.835052
4,0.278200,0.374520,0.831081,0.792244,0.856287,0.737113
5,0.210200,0.473017,0.846847,0.813187,0.870588,0.762887


Validation Results: {'eval_loss': 0.3706875443458557, 'eval_accuracy': 0.8445945945945946, 'eval_f1': 0.8244274809160306, 'eval_precision': 0.8140703517587939, 'eval_recall': 0.8350515463917526, 'eval_runtime': 2.7372, 'eval_samples_per_second': 162.211, 'eval_steps_per_second': 2.557, 'epoch': 5.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task1.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task1.1_hard.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"✅ Submission JSON saved as {output_json_path}")

✅ Submission JSON saved as submission_task1.1_hard.json


In [ ]:
set_seed()
# modelname
modelname = "cardiffnlp/twitter-roberta-base-2022-154m"
# training params
params = {
    "learning_rate": 5e-6,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "num_train_epochs": 10,
    "max_grad_norm": 1.0
}
# pipeline for the subtask1
model2, submission_df2 = sexism_classification_pipeline_task1(EnTrainTask1, EnDevTask1, EnTestTask1, modelname,
2, "single_label_classification", **params )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.676800,0.679320,0.563063,0.000000,0.000000,0.000000
2,0.462800,0.398597,0.817568,0.805755,0.753363,0.865979
3,0.423300,0.367248,0.842342,0.822335,0.810000,0.835052
4,0.279700,0.364829,0.853604,0.826667,0.856354,0.798969
5,0.236300,0.513335,0.835586,0.795518,0.871166,0.731959
6,0.230300,0.530071,0.849099,0.834568,0.800948,0.871134
7,0.143400,0.558783,0.846847,0.818182,0.850000,0.788660
8,0.119500,0.612923,0.860360,0.839378,0.843750,0.835052
9,0.150600,0.635547,0.855856,0.832461,0.845745,0.819588
10,0.079700,0.638284,0.860360,0.838542,0.847368,0.829897


Validation Results: {'eval_loss': 0.6129226088523865, 'eval_accuracy': 0.8603603603603603, 'eval_f1': 0.8393782383419689, 'eval_precision': 0.84375, 'eval_recall': 0.8350515463917526, 'eval_runtime': 2.7847, 'eval_samples_per_second': 159.443, 'eval_steps_per_second': 2.514, 'epoch': 10.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task1.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task1.1_hard2.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task1.1_hard2.json


In [ ]:
set_seed()
# modelname
modelname = "cardiffnlp/twitter-roberta-base-2022-154m"
# training params
params = {
    "learning_rate": 1e-5,
    "per_device_train_batch_size": 8,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "num_train_epochs": 10,
    "max_grad_norm": 1.0
}
# pipeline for the subtask1
model2, submission_df2 = sexism_classification_pipeline_task1(EnTrainTask1, EnDevTask1, EnTestTask1, modelname,
2, "single_label_classification", **params )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.564800,0.384961,0.826577,0.808933,0.779904,0.840206
2,0.418000,0.440494,0.853604,0.832041,0.834197,0.829897
3,0.381600,0.556502,0.858108,0.848921,0.793722,0.912371
4,0.244800,0.635733,0.851351,0.839806,0.793578,0.891753
5,0.141800,0.836029,0.855856,0.827027,0.869318,0.788660


Validation Results: {'eval_loss': 0.556502103805542, 'eval_accuracy': 0.8581081081081081, 'eval_f1': 0.8489208633093526, 'eval_precision': 0.7937219730941704, 'eval_recall': 0.9123711340206185, 'eval_runtime': 2.7498, 'eval_samples_per_second': 161.463, 'eval_steps_per_second': 2.546, 'epoch': 5.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task1.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task1.1_hard3.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task1.1_hard3.json


### Task 2

In [ ]:
# Usage Example
set_seed()

# COMPLETE
params = {
    "learning_rate": 1e-5,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 20,
    "rank": 8,
    "num_train_epochs": 10
              }

model4, submission_df4 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, EnTestTask2, 'bert-base-uncased', 3, "single_label_classification", **params )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.114900,1.063373,0.513699,0.318422,0.288337,0.355912
2,1.003600,0.987601,0.582192,0.245310,0.194064,0.333333
3,0.907700,0.933747,0.582192,0.245310,0.194064,0.333333
4,0.894300,0.870686,0.595890,0.300171,0.366667,0.359715
5,0.738400,0.832550,0.609589,0.375705,0.348659,0.416993
6,0.704200,0.814237,0.650685,0.439047,0.394928,0.496138
7,0.612500,0.812904,0.664384,0.456626,0.515152,0.487246
8,0.453800,0.851198,0.650685,0.542397,0.557420,0.535286
9,0.373700,0.907659,0.650685,0.517334,0.532754,0.517515
10,0.234700,1.040047,0.616438,0.525453,0.523692,0.528037


Validation Results: {'eval_loss': 0.8511983752250671, 'eval_accuracy': 0.6506849315068494, 'eval_f1': 0.542396677081098, 'eval_precision': 0.5574199507389163, 'eval_recall': 0.5352856294032765, 'eval_runtime': 1.0953, 'eval_samples_per_second': 133.293, 'eval_steps_per_second': 2.739, 'epoch': 10.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task2.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task2.1_hard1.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task2.1_hard1.json


In [ ]:
# Usage Example
set_seed()

# COMPLETE
params = {
    "learning_rate": 1e-5,
    "per_device_train_batch_size": 8,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 20,
    "rank": 8,
    "num_train_epochs": 10,
    "max_grad_norm": 1.0
              }

model5, submission_df5 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, EnTestTask2, 'bert-base-uncased', 3, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.024300,1.002686,0.582192,0.246377,0.195402,0.333333
2,0.920300,0.923058,0.582192,0.245310,0.194064,0.333333
3,0.818900,0.832013,0.643836,0.426789,0.384730,0.479857
4,0.712700,0.830897,0.582192,0.458595,0.457057,0.470317
5,0.545200,0.811511,0.664384,0.508917,0.536450,0.517375
6,0.390700,0.916250,0.636986,0.513725,0.514815,0.522031
7,0.394200,1.161094,0.636986,0.544573,0.543462,0.562715
8,0.166000,1.073494,0.712329,0.635477,0.654982,0.624658
9,0.169400,1.221477,0.664384,0.582425,0.580274,0.585617
10,0.055300,1.214341,0.678082,0.597828,0.601407,0.597067


Validation Results: {'eval_loss': 1.0734944343566895, 'eval_accuracy': 0.7123287671232876, 'eval_f1': 0.635477205217616, 'eval_precision': 0.6549820788530466, 'eval_recall': 0.62465834818776, 'eval_runtime': 1.0468, 'eval_samples_per_second': 139.472, 'eval_steps_per_second': 2.866, 'epoch': 10.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task2.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task2.1_hard1.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task2.1_hard1.json


In [ ]:
# Usage Example
set_seed()

# COMPLETE
params = {
    "learning_rate": 5e-6,
    "per_device_train_batch_size": 4,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 20,
    "rank": 4,
    "num_train_epochs": 10,
    "max_grad_norm": 1.0
              }

model6, submission_df6 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, EnTestTask2, 'bert-base-uncased', 3, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.011300,0.971273,0.582192,0.245310,0.194064,0.333333
2,0.888800,0.894989,0.589041,0.266500,0.363426,0.343434
3,0.763400,0.815217,0.630137,0.407462,0.370609,0.453476
4,0.612600,0.822999,0.616438,0.486183,0.492965,0.502283
5,0.412200,0.874489,0.664384,0.445221,0.414141,0.485443
6,0.377600,0.961347,0.650685,0.533495,0.542441,0.537858
7,0.412400,1.058593,0.643836,0.489851,0.498299,0.503807
8,0.185200,1.118459,0.650685,0.523567,0.528445,0.529874
9,0.198000,1.152980,0.671233,0.558008,0.567146,0.557605
10,0.287400,1.177065,0.650685,0.544370,0.547619,0.545841


Validation Results: {'eval_loss': 1.1529804468154907, 'eval_accuracy': 0.6712328767123288, 'eval_f1': 0.5580084278583177, 'eval_precision': 0.5671460434768135, 'eval_recall': 0.5576054664289959, 'eval_runtime': 1.0925, 'eval_samples_per_second': 133.632, 'eval_steps_per_second': 2.746, 'epoch': 10.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task2.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task2.1_hard3.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task2.1_hard3.json


### Task 3

In [ ]:
!pip install -U PyEvALL

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import random
import os
import pandas as pd
import sys
import tempfile
import time

#Importing the required modules to use the ICM measure

from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory
from pyevall.reports.reports import PyEvALLReport
from pyevall.utils.utils import PyEvALLUtils

from functools import partial

In [ ]:
def ICMWrapper(pred, labels, multi=False,ids=None):
    test = PyEvALLEvaluation()
    metrics=[MetricFactory.ICM.value]
    params= dict()
    fillLabel=None
    if multi:
        params[PyEvALLUtils.PARAM_REPORT]="embedded"
        hierarchy={"True":['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE', 'MISOGYNY-NON-SEXUAL-VIOLENCE', 'OBJECTIFICATION', 'SEXUAL-VIOLENCE'],
        "False":[]}
        params[PyEvALLUtils.PARAM_HIERARCHY]=hierarchy
        fillLabel = lambda x: ["False"] if len(x)== 0 else x
    else:
        params[PyEvALLUtils.PARAM_REPORT]="simple"
        fillLabel = lambda x: str(x)


    truth_name, predict_name=None, None
    if ids is None:
        ids=list(range(len(labels)))

    with tempfile.NamedTemporaryFile(mode='w', delete=False, encoding='utf-8') as truth:
        truth_name=truth.name
        truth_df=pd.DataFrame({'test_case': ['EXIST2025']*len(labels),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in labels]})
        if multi==True:
            truth_df=truth_df.astype('object')
        truth.write(truth_df.to_json(orient="records"))

    with  tempfile.NamedTemporaryFile(mode='w', delete=False) as predict:
        predict_name=predict.name
        predict_df=pd.DataFrame({'test_case': ['EXIST2025']*len(pred),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in pred]})
        if multi==True:
            predict_df=predict_df.astype('object')
        predict.write(predict_df.to_json(orient="records"))

    report = test.evaluate(predict_name, truth_name, metrics, **params)
    os.unlink(truth_name)
    os.unlink(predict_name)

    icm = None
    if 'metrics' in report.report:
        if 'ICM' in report.report["metrics"]: icm=float(report.report["metrics"]['ICM']["results"]["average_per_test_case"])
    return icm


In [ ]:
class SexismDatasetMulti(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)}

In [ ]:
def compute_metrics_3(pred, lencoder):
    labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    preds = torch.sigmoid(torch.tensor(pred.predictions)).numpy()
    preds_binary = (preds >= 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds_binary, average=None, zero_division=0
    )
    acc = accuracy_score(labels, preds_binary)
    icm= ICMWrapper(lencoder.inverse_transform(preds_binary), lencoder.inverse_transform(labels), multi=True)
    # Macro averages
    precision_macro = np.mean(precision)
    recall_macro = np.mean(recall)
    f1_macro = np.mean(f1)
    metrics = {}
    metrics.update({
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'ICM': icm
    })
    return metrics

In [ ]:
def sexism_classification_pipeline_task3(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=5, ptype="multi_label_classification", **args):
    # Model and Tokenizer
    labelEnc= MultiLabelBinarizer()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype)

    # Prepare datasets
    train_dataset = SexismDatasetMulti(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDatasetMulti(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"ICM")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #compute_metrics=compute_metrics_3,
        compute_metrics = partial(compute_metrics_3, lencoder=labelEnc),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    if testInfo is not None:
      # Prepare test dataset for prediction
      test_dataset = SexismDatasetMulti(testInfo[1], [[0,0,0,0,0]] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

      # Predict test set labels
      predictions = trainer.predict(test_dataset)
      #predicted_labels = np.argmax(predictions.predictions, axis=1)
      predicted_probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
      predicted_labels = (predicted_probs >= 0.5).astype(int)

      # Create submission DataFrame
      submission_df = pd.DataFrame({
          'id': testInfo[0],
          'label': labelEnc.inverse_transform(predicted_labels),
          "test_case": ["EXIST2025"]*len(predicted_labels)

      })
      submission_df.to_csv('sexism_predictions_task3.csv', index=False)
      print("Prediction TASK3 completed. Results saved to sexism_predictions_task2.csv")
      return model, submission_df
    return model, eval_results

In [ ]:
# COMPLETE

# Usage Example
set_seed()

# COMPLETE

modelname = "bert-base-uncased"
params = {
    "num_train_epochs": 8,
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model4, eval_results4 = sexism_classification_pipeline_task3(EnTrainTask3, EnDevTask3, EnTestTask3, model_name=modelname, nlabels=5, ptype="multi_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.682900,0.686357,0.364235,0.230678,0.208318,-1.623546
2,0.664700,0.654581,0.675177,0.594807,0.534406,-0.416418
3,0.625400,0.628493,0.681803,0.612875,0.608438,-0.225769
4,0.588700,0.603674,0.681841,0.656998,0.665740,-0.625157
5,0.552400,0.586857,0.697012,0.649916,0.669036,-0.547275
6,0.516900,0.586887,0.743744,0.600159,0.655427,-0.894956
7,0.484000,0.589782,0.736212,0.613109,0.666690,-1.118215


2025-05-11 12:53:01,167 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:53:01,234 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
cargado 29
2025-05-11 12:53:53,421 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:53:53,485 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 12:54:48,214 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:54:48,274 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 12:55:49,578 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:55:49,643 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 12:56:43,053 - pyevall.evaluation 

2025-05-11 12:58:52,775 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:58:52,888 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.628493070602417, 'eval_precision_macro': 0.6818031212344986, 'eval_recall_macro': 0.6128754449223017, 'eval_f1_macro': 0.6084384598604338, 'eval_ICM': -0.22576947200286962, 'eval_runtime': 2.8276, 'eval_samples_per_second': 119.535, 'eval_steps_per_second': 2.122, 'epoch': 7.0}
2025-05-11 12:59:00,082 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 12:59:00,524 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task3.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task3.1_hard1.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task3.1_hard1.json


In [ ]:
# COMPLETE

# Usage Example
set_seed()

# COMPLETE

modelname = "cardiffnlp/twitter-roberta-base"
params = {
    "num_train_epochs": 8,
    "learning_rate": 5e-6,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model8, eval_results8 = sexism_classification_pipeline_task3(EnTrainTask3, EnDevTask3, EnTestTask3, model_name=modelname, nlabels=5, ptype="multi_label_classification", **params )

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.686200,0.682422,0.504648,0.508592,0.452731,-0.769239
2,0.661700,0.663089,0.377383,0.560000,0.450107,-0.607392
3,0.637700,0.610423,0.710489,0.631594,0.631484,-0.097623
4,0.571000,0.582288,0.703971,0.678681,0.681598,-0.037649
5,0.532900,0.572034,0.732247,0.654685,0.686488,-0.502648
6,0.521800,0.568632,0.747008,0.652639,0.691282,-0.563348
7,0.509300,0.564616,0.747605,0.670103,0.702692,-0.393293
8,0.490300,0.565404,0.749383,0.666863,0.702968,-0.494477


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

2025-05-11 13:02:10,829 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:02:10,931 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:03:32,994 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:03:33,072 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:05:17,355 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:05:17,437 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:06:48,797 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:06:48,904 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:08:17,394 - pyevall.evaluation - INFO -   

2025-05-11 13:12:15,483 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:12:15,586 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5822882056236267, 'eval_precision_macro': 0.7039713092500897, 'eval_recall_macro': 0.6786813273167561, 'eval_f1_macro': 0.6815984524571663, 'eval_ICM': -0.03764925360773205, 'eval_runtime': 2.6193, 'eval_samples_per_second': 129.04, 'eval_steps_per_second': 2.291, 'epoch': 8.0}
2025-05-11 13:12:22,107 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:12:22,251 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task3.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task3.1_hard2.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task3.1_hard2.json


In [ ]:
# COMPLETE

# Usage Example
set_seed()

# COMPLETE

modelname = "cardiffnlp/twitter-roberta-base"
params = {
    "num_train_epochs": 10,
    "learning_rate": 1e-6,
    "per_device_train_batch_size": 8,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model9, sub_df9 = sexism_classification_pipeline_task3(EnTrainTask3, EnDevTask3, EnTestTask3, model_name=modelname, nlabels=5, ptype="multi_label_classification", **params )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.683900,0.684598,0.493407,0.404031,0.407468,-1.071593
2,0.670900,0.674848,0.370414,0.584615,0.453452,-0.572176
3,0.665300,0.656880,0.394116,0.540513,0.451236,-0.605045
4,0.642600,0.639033,0.538363,0.543736,0.461411,-0.559553
5,0.610200,0.621736,0.732863,0.578327,0.537018,-0.370058
6,0.629500,0.612433,0.719013,0.622084,0.613085,-0.144411
7,0.581700,0.604449,0.699356,0.649784,0.641202,-0.049899
8,0.589400,0.600620,0.701882,0.660261,0.655595,0.005650
9,0.589700,0.598328,0.700203,0.664790,0.660591,0.022273
10,0.584100,0.597602,0.700269,0.664790,0.660966,0.023989


2025-05-11 13:14:28,406 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:14:28,468 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:15:36,857 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:15:36,924 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:16:41,368 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:16:41,433 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:17:56,026 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:17:56,089 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-11 13:19:10,311 - pyevall.evaluation - INFO -   

2025-05-11 13:25:26,727 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:25:26,809 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5976016521453857, 'eval_precision_macro': 0.70026902482249, 'eval_recall_macro': 0.6647901200114381, 'eval_f1_macro': 0.6609660642286068, 'eval_ICM': 0.02398921494710741, 'eval_runtime': 2.4534, 'eval_samples_per_second': 137.769, 'eval_steps_per_second': 2.446, 'epoch': 10.0}
2025-05-11 13:25:33,155 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-11 13:25:33,293 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task3.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task3.1_hard3.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task3.1_hard3.json


### Spanish

In [ ]:
set_seed()
# modelname
modelname = "dccuchile/bert-base-spanish-wwm-cased"
# training params
params = {
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "num_train_epochs": 10
}
# pipeline for the subtask1
model10, submission_df10 = sexism_classification_pipeline_task1(SpTrainTask1, SpDevTask1, SpTestTask1, modelname,
2, "single_label_classification", **params )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.658800,0.625216,0.677551,0.752351,0.636605,0.919540
2,0.455000,0.416639,0.804082,0.828571,0.775920,0.888889
3,0.361800,0.383404,0.826531,0.827586,0.879310,0.781609
4,0.234300,0.398693,0.848980,0.854902,0.875502,0.835249
5,0.188000,0.508999,0.812245,0.814516,0.859574,0.773946
6,0.147600,0.755201,0.816327,0.806034,0.921182,0.716475


Validation Results: {'eval_loss': 0.3986932635307312, 'eval_accuracy': 0.8489795918367347, 'eval_f1': 0.8549019607843137, 'eval_precision': 0.8755020080321285, 'eval_recall': 0.8352490421455939, 'eval_runtime': 3.2477, 'eval_samples_per_second': 150.876, 'eval_steps_per_second': 2.463, 'epoch': 6.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task1.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task1.1_hard1_sp.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task1.1_hard1_sp.json


### Task 2

In [ ]:
# Usage Example
set_seed()

# COMPLETE
params = {
    "learning_rate": 5e-6,
    "per_device_train_batch_size": 4,
    "weight_decay": 0.01,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 20,
    "rank": 4,
    "num_train_epochs": 10,
    "max_grad_norm": 1.0
              }

model11, submission_df11 = sexism_classification_pipeline_task2(SpTrainTask2, SpDevTask2, SpTestTask2, "dccuchile/bert-base-spanish-wwm-cased", 3, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.811500,0.954038,0.560386,0.239422,0.186795,0.333333
2,0.782500,0.913635,0.594203,0.442640,0.550069,0.457319
3,0.678400,1.008406,0.632850,0.521566,0.604906,0.504215
4,0.506200,1.078372,0.661836,0.564172,0.630099,0.559607
5,0.489800,1.172615,0.637681,0.566013,0.578603,0.558159
6,0.260700,1.498127,0.618357,0.541952,0.549586,0.539272
7,0.607800,1.671597,0.603865,0.512940,0.524411,0.507015
8,0.197200,1.757962,0.623188,0.552850,0.552903,0.553133
9,0.217100,1.869030,0.623188,0.544084,0.549247,0.542146


Validation Results: {'eval_loss': 1.1726150512695312, 'eval_accuracy': 0.6376811594202898, 'eval_f1': 0.5660130718954249, 'eval_precision': 0.5786026776301919, 'eval_recall': 0.5581586657651566, 'eval_runtime': 1.4585, 'eval_samples_per_second': 141.929, 'eval_steps_per_second': 2.743, 'epoch': 9.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task2.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task2.1_hard1_sp.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task2.1_hard1_sp.json


### Task 3

In [ ]:
# COMPLETE

# Usage Example
set_seed()

# COMPLETE

# modelname = "cardiffnlp/twitter-roberta-base"
params = {
    "num_train_epochs": 8,
    "learning_rate": 5e-6,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model12, submission_df12 = sexism_classification_pipeline_task3(SpTrainTask3, SpDevTask3, SpTestTask3, "dccuchile/bert-base-spanish-wwm-cased", nlabels=5, ptype="multi_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.668200,0.672054,0.477863,0.659492,0.540486,-0.381575
2,0.642400,0.654670,0.507737,0.645858,0.563621,-0.283012
3,0.605500,0.605383,0.703700,0.705745,0.665033,-0.058899
4,0.549900,0.575112,0.689213,0.748147,0.709726,0.092357
5,0.527400,0.573010,0.710131,0.706735,0.698840,-0.079510
6,0.493900,0.565412,0.722810,0.724429,0.717820,-0.045740
7,0.507500,0.569421,0.729562,0.700782,0.706873,-0.185192
8,0.486200,0.570195,0.725323,0.710041,0.710278,-0.074547


2025-05-16 09:38:33,554 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:38:33,642 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
cargado 29
2025-05-16 09:39:45,390 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:39:45,460 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-16 09:40:52,036 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:40:52,104 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-16 09:41:58,309 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:41:58,420 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-16 09:43:50,372 - pyevall.evaluation 

2025-05-16 09:48:50,008 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:48:50,129 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5751122832298279, 'eval_precision_macro': 0.68921256250568, 'eval_recall_macro': 0.7481465607412243, 'eval_f1_macro': 0.7097263561363605, 'eval_ICM': 0.09235712003805847, 'eval_runtime': 3.0879, 'eval_samples_per_second': 127.271, 'eval_steps_per_second': 2.267, 'epoch': 8.0}
2025-05-16 09:48:57,821 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-16 09:48:57,991 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task2.csv


In [ ]:
import pandas as pd
import json

# Load your model output CSV
csv_path = "sexism_predictions_task3.csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": row["id"],
        "value": row["label"],  # Must be "YES" or "NO"
        "test_case": row["test_case"]
    })

# Save as JSON file
output_json_path = "submission_task3.1_hard1_sp.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"Submission JSON saved as {output_json_path}")

Submission JSON saved as submission_task3.1_hard1_sp.json


# Corrected json generation file

In [ ]:
# Corrected
import pandas as pd
import json

# Load your model output CSV
csv_path = "/content/csvs/sexism_predictions_task3 (2).csv"
df = pd.read_csv(csv_path)

# Convert to required JSON structure
submission = []
for _, row in df.iterrows():
    submission.append({
        "id": str(row["id"]),  # Ensure 'id' is a string
        "value": row["label"].strip().upper(),  # Ensure valid format like "YES" or "NO"
        "test_case": row["test_case"].strip()
    })

# Ensure correct number of instances
if len(submission) != 1053:
    print(f"⚠️ Warning: You have {len(submission)} instances, expected 1053 for Task2_1.")

# Save as JSON file with proper naming convention (e.g., task2_1_teamname_lang_run.json)
output_json_path = "/content/runs_new/task1_3_hard_Güeypingüino_2.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)

print(f"✅ Submission JSON saved as {output_json_path}")


⚠️ Warning: You have 978 instances, expected 1053 for Task2_1.
✅ Submission JSON saved as /content/runs_new/task1_3_hard_Güeypingüino_2.json


In [ ]:
import json

# Paths to your JSON files
file1 = '/content/runs_new/task1_1_hard_Güeypingüino_2.json'
file2 = '/content/runs_new/task1_1_hard_Güeypingüino_3.json'
output_file = 'task1_1_hard_Güeypingüino_2.json'

# Load JSON arrays from both files
with open(file1, 'r', encoding='utf-8') as f1:
    data1 = json.load(f1)

with open(file2, 'r', encoding='utf-8') as f2:
    data2 = json.load(f2)

# Check that both files contain arrays
if not isinstance(data1, list) or not isinstance(data2, list):
    raise ValueError("Both files must contain JSON arrays.")

# Concatenate the arrays
combined = data1 + data2

# Save to a new JSON file
with open(output_file, 'w', encoding='utf-8') as fout:
    json.dump(combined, fout, indent=2, ensure_ascii=False)

print(f"✅ Combined JSON saved as: {output_file}")


✅ Combined JSON saved as: task1_1_hard_Güeypingüino_2.json


### Validation script

In [ ]:
'''
Created on 1 may 2024

@author: jcalbornoz
'''
from os import listdir
from os.path import isfile, join
import json
import jsonschema
from jsonschema import validate

ID= "id"
TEST_CASE="test_case"
VALUE = "value"
TASK1_1="task1_1"
TASK1_2="task1_2"
TASK1_3="task1_3"
TASK2_1="task2_1"
TASK2_2="task2_2"
TASK2_3="task2_3"
TASK3_1="task2_1"
TASK3_2="task2_2"
TASK3_3="task2_3"

LIST_LABELS_SUBTASK1=["NO", "YES"]
LIST_LABELS_SUBTASK2=["NO", "REPORTED", "JUDGEMENTAL", "DIRECT"]
LIST_LABELS_SUBTASK3=["NO", "IDEOLOGICAL-INEQUALITY", "STEREOTYPING-DOMINANCE", "MISOGYNY-NON-SEXUAL-VIOLENCE", "SEXUAL-VIOLENCE", "OBJECTIFICATION"]
LIST_LABELS_SUBTASK2_2=["NO", "JUDGEMENTAL", "DIRECT"]

FORMAT_JSON_SCHEMA= {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "test_case": {"type": "string"},
            "id":{"type": "string"},
            "value": {
                "anyOf": [
                    {"type": "string"},
                    {"type": "array", "items": {"type": "string"},"minItems": 1},
                    {"type": "integer"},
                    {
                        "type": "object",
                        "patternProperties": {
                        "^.*$": {"type": "number"},    }
                    },
                ]
            },
        },
        "required": ["test_case", "id", "value"],
        "additionalProperties": False
    },

}

def read_files_by_task(path):
    return [f for f in listdir(path) if isfile(join(path, f))]


def parser_json(path):
    data = None
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except ValueError as e:
        print(e)
        return False

    try:
        validate(instance=data, schema=FORMAT_JSON_SCHEMA)
    except jsonschema.exceptions.ValidationError as e:
        print("Errors found in the file: ", path , "\n", e)
        return False

    return True


def process_format_runs_by_task(task_folder):
    onlyfiles = read_files_by_task(task_folder)
    for file in onlyfiles:
        print("******************************************************")
        print("Analizing file "+ file)
        run = task_folder + file
        if not parser_json(run):
            continue

        with open(run, 'r', encoding='utf-8') as f:
            data = json.load(f)

            tasks= file.split("_")
            if len(tasks)!=5:
                print("Error, the name of the run ", run, "is incorrect. Please check the Lab guidelines.")
                continue
            task= tasks[0] + "_" + tasks[1]
            data_es=dict()
            data_en=dict()
            #Check size test data
            if task==TASK1_1 or task==TASK1_2 or task==TASK1_3:
                if len(data)!= 2076:
                    print("Error in the number of instances in Tweets test set: ", len(data), " . There must be 2076 instances.")
            elif task==TASK2_1 or task==TASK2_2 or task==TASK2_3:
                if len(data)!= 1053:
                    print("Error in the number of instances in Memes test set: ", len(data), " . There must be 1053 instances.")
            elif task==TASK3_1 or task==TASK3_2 or task==TASK3_3:
                if len(data)!= 674:
                    print("Error in the number of instances in Videos test set: ", len(data), " . There must be 674 instances.")

            for instance in data:

                for property in instance:
                    #Check proporties in json object
                    if property==ID or property==TEST_CASE:
                        continue

                    elif property==VALUE:
                        if type(instance[VALUE])==type(""):
                            #Check labels
                            if task==TASK1_1 or task==TASK2_1 or task==TASK3_1:
                                if not instance[VALUE] in LIST_LABELS_SUBTASK1:
                                    print("ERROR in label for task ", task, " format ", VALUE, " hard label ", instance[VALUE])

                            elif task==TASK1_2:
                                if not instance[VALUE] in LIST_LABELS_SUBTASK2:
                                    print("ERROR in label for task ", task, " format ", VALUE, " hard  label ", instance[VALUE])

                            elif task==TASK2_2 or task==TASK3_2:
                                if not instance[VALUE] in LIST_LABELS_SUBTASK2_2:
                                    print("ERROR in label for task ", task, " format ", VALUE, " hard  label ", instance[VALUE])

                        elif type(instance[VALUE])==type([]):
                            if task==TASK1_3 or task==TASK2_3 or task==TASK3_3 :
                                labels = instance[VALUE]
                                for label in labels:
                                    if not label in LIST_LABELS_SUBTASK3:
                                        print("ERROR in label for task ", task, " format ", VALUE, " hard  label ", instance[VALUE])

                        elif type(instance[VALUE])==type(dict()):
                            if task==TASK1_1 or task==TASK2_1 or task==TASK3_1:
                                labels = instance[VALUE]
                                if len(labels)!=2:
                                    print("ERROR in label for task ", task, " format ", VALUE, " wrong number ", len(labels))
                                x=0
                                for label in labels:
                                    x+=float(labels[label])
                                    if not label in LIST_LABELS_SUBTASK1:
                                        print("ERROR in label for task ", task, " format ", VALUE, "soft label ", instance[VALUE])
                                if x>1.001:
                                    print("ERROR in label for task ", task, " format ", VALUE, " sum is bigger than 1.0 ", x)

                            elif task==TASK1_2:
                                labels = instance[VALUE]
                                if len(labels)!=4:
                                    print("ERROR in label for task ", task, " format ", VALUE, " wrong number ", len(labels))
                                x=0
                                for label in labels:
                                    x+=float(labels[label])
                                    if not label in LIST_LABELS_SUBTASK2:
                                        print("ERROR in label for task ", task, " format ", VALUE, "soft  label ", instance[VALUE])
                                if x>1.001:
                                    print("ERROR in label for task ", task, " format ", VALUE, " sum is bigger than 1.0 ", x)

                            elif task==TASK2_2 or task==TASK3_2:
                                labels = instance[VALUE]
                                if len(labels)!=3:
                                    print("ERROR in label for task ", task, " format ", VALUE, " wrong number ", len(labels))
                                x=0
                                for label in labels:
                                    x+=float(labels[label])
                                    if not label in LIST_LABELS_SUBTASK2_2:
                                        print("ERROR in label for task ", task, " format ", VALUE, "soft  label ", instance[VALUE])
                                if x>1.001:
                                    print("ERROR in label for task ", task, " format ", VALUE, " sum is bigger than 1.0 ", x)

                            elif task==TASK1_3 or task==TASK2_3 or task==TASK3_3 :
                                labels = instance[VALUE]
                                if len(labels)!=6:
                                    print("ERROR in label for task ", task, " format ", VALUE, " wrong number ", len(labels))
                                for label in labels:
                                    if not label in LIST_LABELS_SUBTASK3:
                                        print("ERROR in label for task ", task, " format ", VALUE, "soft  label ", instance[VALUE])
                        else:
                            print("Error format value property.")

                    else:
                        print("ERROR in json format , property not allowed ", property)


                if task==TASK1_1 or task==TASK1_2 or task==TASK1_3:
                    if int(instance[ID])>=500001 and int(instance[ID])<=501098:
                        data_es[instance[ID]]=instance
                    elif int(instance[ID])>=600001 and int(instance[ID])<=600978:
                        data_en[instance[ID]]=instance
                    else:
                        print("ERROR en ids ", id)
                elif task==TASK2_1 or task==TASK2_2 or task==TASK2_3:
                    if int(instance[ID])>=310001 and int(instance[ID])<=310540:
                        data_es[instance[ID]]=instance
                    elif int(instance[ID])>=410001 and int(instance[ID])<=410513:
                        data_en[instance[ID]]=instance
                    else:
                        print("ERROR en ids ", instance[ID])
                elif task==TASK3_1 or task==TASK3_2 or task==TASK3_3:
                    if int(instance[ID])>=320001 and int(instance[ID])<=320304:
                        data_es[instance[ID]]=instance
                    elif int(instance[ID])>=420001 and int(instance[ID])<=420370:
                        data_en[instance[ID]]=instance
                    else:
                        print("ERROR en ids ", instance[ID])


            if task==TASK1_1 or task==TASK1_2 or task==TASK1_3:
                if len(data_es)!=1098:
                    print("ERROR number of instances in the Tweets Spanish dataset ", len(data_es))
                if len(data_en)!=978:
                    print("ERROR  number of instances in the Tweets English dataset ", len(data_en))
            elif task==TASK2_1 or task==TASK2_2 or task==TASK2_2:
                if len(data_es)!=540:
                    print("ERROR  number of instances in the Memes Spanish dataset ", len(data_es))
                if len(data_en)!=513:
                    print("ERROR  number of instances in the Memes English dataset ", len(data_en))
            elif task==TASK3_1 or task==TASK3_2 or task==TASK3_2:
                if len(data_es)!=304:
                    print("ERROR  number of instances in the Videos Spanish dataset ", len(data_es))
                if len(data_en)!=370:
                    print("ERROR  number of instances in the Videos English dataset ", len(data_en))

            print("******************************************************")


if __name__ == '__main__':
    # process_format_runs_by_task("/content/task1_1_hard_Güey_pingüino_1.json")
    process_format_runs_by_task("/content/")

******************************************************
Analizing file task1_3_hard_Güeypingüino_2.json
******************************************************
******************************************************
Analizing file task1_3_hard_Güeypingüino_1.json
******************************************************
******************************************************
Analizing file task1_1_hard_Güeypingüino_2.json
******************************************************
******************************************************
Analizing file task1_2_hard_Güeypingüino_1.json
******************************************************
******************************************************
Analizing file task1_1_hard_Güeypingüino_1.json
******************************************************
******************************************************
Analizing file task1_2_hard_Güeypingüino_2.json
******************************************************
